In [1]:
%load_ext autoreload
%autoreload 2
from CW import CW
from FGSM import FGSM
from OPT_attack import OPT_attack
from OPT_genattack import OPT_genattack
from ZOO import ZOO
from models import PytorchModel
import torch
from allmodels import MNIST, load_model, load_mnist_data, load_cifar10_data, CIFAR10
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [2]:
net = MNIST()
net.cuda()
net = torch.nn.DataParallel(net, device_ids=[0])
load_model(net,'mnist_gpu.pt')
net.eval()
model = net.module if torch.cuda.is_available() else net

In [3]:
amodel = PytorchModel(model, bounds=[0,1], num_classes=10)
attack = CW(amodel)

In [ ]:
attack = FGSM(amodel)

In [ ]:
attack = OPT_attack(amodel)

In [ ]:
attack = OPT_genattack(amodel)

In [4]:
attack = ZOO(amodel)

In [5]:
train_loader, test_loader, train_dataset, test_dataset = load_mnist_data()
for i, (xi,yi) in enumerate(test_loader):
    if i==1:
        break
    #xi_v=torch.autograd.Variable(xi)
    #print(xi.size())
    #res= amodel.predict(xi)
    #_,label=torch.max(res,1)
    #print(label)
#print(res.size())

In [9]:
attack(xi,yi)

26.693628311157227 0.0 26.693628311157227
26.21864128112793 3.220593924879722e-08 26.21864128112793
25.42763328552246 8.552288477403636e-08 25.42763328552246
24.496135711669922 9.179401416758992e-08 24.496135711669922
23.294424057006836 4.5905967915871315e-09 23.294424057006836
22.42807960510254 5.786674250884971e-08 22.42807960510254
20.79254150390625 1.108426772589155e-06 20.792539596557617
19.1235294342041 2.5742863840605423e-08 19.1235294342041
17.889942169189453 2.4088260275334505e-08 17.889942169189453
16.84569549560547 4.0981802840178716e-08 16.84569549560547
15.98200798034668 4.845671952580233e-08 15.98200798034668
14.947929382324219 6.968761678827207e-10 14.947929382324219
13.801546096801758 5.6355635713600805e-09 13.801546096801758
13.093856811523438 5.261470050754724e-07 13.093855857849121
12.482667922973633 6.49138698349816e-08 12.482667922973633
11.934165954589844 1.313876509811962e-07 11.934165954589844
11.36858081817627 1.9959182395723474e-07 11.36858081817627
10.7860364

KeyboardInterrupt: 

In [ ]:
from torch.autograd import Variable
target = torch.LongTensor(1).random_(10)
attack(xi,target,TARGETED=True)

In [ ]:
xi=xi.numpy()
attack(xi,yi)

In [ ]:
idx_a = [6312, 6891, 664, 4243, 8377, 7962, 6635, 4970, 7809, 5867, 9559, 3579, 8269, 2282, 4618, 2290, 1554, 4105, 8726, 9862, 2408, 5082, 1619, 1209, 5410, 7736, 9172, 1650, 5797, 7114, 5181, 3351, 9053, 7816, 7254, 8542, 4268, 1021, 8990, 231, 1529, 6535, 19, 8087, 5459, 3997, 5329, 1032, 3131, 9299, 3633, 3910, 2335, 8897, 7340, 1495, 1319, 5244, 8323, 8017, 1787, 4939, 9032, 4770, 2045, 8970, 5452, 8853, 3330, 9883, 8966, 9628, 4713, 7291, 1502, 9770, 6307, 5195, 9432, 3967, 4757, 3013, 3103, 3060, 541, 4261, 7808, 1132, 1472, 2134, 2451, 634, 1315, 8858, 6411, 8595, 4516, 8550, 3859, 3526]
res=[]
fail = 0
#idx_a = [6312]
for idx in idx_a:
    count = 0
    #random_all = torch.zeros(1,1,28,28)
    print(idx)
    xi, yi = test_dataset[idx]
    #xi = xi.numpy()
    #yi = yi.cuda()
    for _ in range(30):
        xi = xi.expand(10000,1,28,28)
    #for _ in range(130000):
        random_noise = torch.randn(10000,1,28,28)
        norm_r = torch.norm(torch.norm(torch.norm(random_noise,2,1),2,1),2,1)
        norm_r = norm_r.unsqueeze(1).unsqueeze(2).expand(xi.size()[0],1,28).unsqueeze(3).expand(xi.size()[0],1,28,28)
    #print(norm_r)
        random_noise/=norm_r
    #print(random_noise)
    #random_all = torch.cat((random_all,random_noise),0)
    #random_all =random_all[1:,:,:,:]
    #print(random_all)
    #random_noise = random_noise.expand(1000,1,28,28)
    #random_noise = torch.autograd
        new_xi = xi + random_noise
        res= amodel.predict(new_xi)
        _, p_label = torch.max(res,1)
    #print(p_label,yi)
        right = (p_label.data!=yi).nonzero()
        if len(right.size())!=0:
            count+=1
        
    if count==0:
        fail +=1
#print(res)

In [ ]:
print(fail) 

In [ ]:
print(len(idx_a))

In [ ]:
net = CIFAR10()
net.cuda()
net = torch.nn.DataParallel(net, device_ids=[0])
load_model(net,'cifar10_gpu.pt')
net.eval()
model = net.module if torch.cuda.is_available() else net
train_loader, test_loader, train_dataset, test_dataset = load_cifar10_data()
amodel = PytorchModel(model, bounds=[0,1], num_classes=10)
idx_a =  [6311, 6890, 663, 4242, 8376, 7961, 6634, 4969, 7808, 5866, 9558, 3578, 8268, 2281, 4617, 2289, 1553, 4104, 8725, 9861, 2407, 5081, 1618, 1208, 5409, 7735, 9171, 1649, 5796,7113, 5180, 3350, 9052, 7815, 7253, 8541, 4267, 1020, 8989, 230, 1528, 6534, 18, 8086, 5458, 3996, 5328, 1031, 3130, 9298, 3632, 3909, 2334, 8896, 7339, 1494, 1318, 5243, 8322, 8016, 1786, 4938, 9031, 4769, 2044, 8969, 5451, 8852, 3329, 9882, 8965, 9627, 4712, 7290, 1501, 9769, 6306, 5194, 9431, 3966, 4756, 3012, 3102, 3059, 540, 4260, 7807, 1131, 1471, 2133, 2450, 633, 1314, 8857, 6410, 8594, 4515, 8549, 3858, 3525, 6411, 4360, 7753, 7413, 684, 3343, 6785, 7079, 2263]
fail = 0
#idx_a = [6311]
for idx in idx_a:
    #random_all = torch.zeros(1,1,28,28)
    count =0
    print(idx)
    xi, yi = test_dataset[idx]
    #xi = xi.numpy()
    #yi = yi.cuda()
    xi = xi.expand(10000,3,32,32)
    for _ in range(30):
    #for _ in range(130000):
        random_noise = torch.randn(10000,3,32,32)
        norm_r = torch.norm(torch.norm(torch.norm(random_noise,2,1),2,1),2,1)
        norm_r = norm_r.unsqueeze(1).unsqueeze(2).expand(xi.size()[0],3,32).unsqueeze(3).expand(xi.size()[0],3,32,32)
    #print(norm_r)
        random_noise/=norm_r
        random_noise *= 0.15
    #print(random_noise)
    #random_all = torch.cat((random_all,random_noise),0)
    #random_all =random_all[1:,:,:,:]
    #print(random_all)
    #random_noise = random_noise.expand(1000,1,28,28)
    #random_noise = torch.autograd
        new_xi = xi + random_noise
        res= amodel.predict(new_xi)
        _, p_label = torch.max(res,1)
    #print(p_label,yi)
        right = (p_label.data!=yi).nonzero()
        if len(right.size())!=0:
            count+=1
        
    if count==0:
        fail +=1

In [ ]:
print(fail,count) 

In [ ]:
print(len(idx_a))